# 📊 EDA - Parte 2: Análise de Receitas e Despesas

## Análise Detalhada para Área Financeira

**Objetivo:** Explorar padrões de receita e despesa para insights de negócio.

---

### 📋 Sumário
1. Receitas por Unidade e Tipo
2. Comparativo Real vs Orçado
3. Despesas por Pacote
4. Análise de Variações

In [ ]:
# Imports e Configurações
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

EXCEL_PATH = r'..\01_dados_originais\dados_case_pbi.xlsx'
MESES = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']

COLORS = {
    'realizado': '#2ecc71',
    'orcado': '#3498db',
    'sales': '#e74c3c',
    'service': '#9b59b6'
}

print("✅ Configuração carregada")

In [ ]:
# Carregar dados das despesas (formato tabular)
df_desp_real = pd.read_excel(EXCEL_PATH, sheet_name='Despesas_Realizado')
df_desp_orc = pd.read_excel(EXCEL_PATH, sheet_name='Despesas_Orç')

print(f"📊 Despesas Realizado: {df_desp_real.shape}")
print(f"📊 Despesas Orçado: {df_desp_orc.shape}")
print(f"\nColunas: {df_desp_real.columns.tolist()}")

---
## 1️⃣ Análise de Receitas

**Nota:** Os dados de receita estão em formato REPORT e foram pré-processados.

In [ ]:
# Carregar receita processada (se disponível) ou analisar raw
xlsx = pd.ExcelFile(EXCEL_PATH)
df_receita_raw = pd.read_excel(xlsx, sheet_name='Receita_Realizado', header=None)

print("📊 ESTRUTURA DA RECEITA REALIZADO (RAW)")
print("="*60)
print(f"Shape: {df_receita_raw.shape}")
print("\nPrimeiras linhas (headers):")
display(df_receita_raw.head(5))

In [ ]:
# Identificar tipos de receita (SALES vs SERVICE)
# Coluna 0 contém os tipos
tipos_receita = df_receita_raw.iloc[3:, 0].dropna().unique()

print("🏷️ TIPOS DE RECEITA IDENTIFICADOS:")
for tipo in tipos_receita[:10]:  # Limitar exibição
    if pd.notna(tipo) and str(tipo).strip():
        print(f"  • {tipo}")

In [ ]:
# Analisar totais de receita (linha 1 geralmente tem totais mensais)
totais_mensais = df_receita_raw.iloc[1, 1:13]
total_anual = df_receita_raw.iloc[1, 13] if df_receita_raw.shape[1] > 13 else totais_mensais.sum()

print("💰 RECEITA REALIZADO - TOTAIS MENSAIS")
print("="*60)
for i, (mes, valor) in enumerate(zip(MESES, totais_mensais)):
    if pd.notna(valor) and isinstance(valor, (int, float)):
        print(f"  {mes}: R$ {valor:>12,.2f}")

if pd.notna(total_anual):
    print(f"\n  TOTAL ANUAL: R$ {total_anual:,.2f}")

---
## 2️⃣ Análise de Despesas

In [ ]:
# Estrutura das despesas
print("📊 ESTRUTURA DAS DESPESAS")
print("="*60)
print(f"\nColunas: {df_desp_real.columns.tolist()}")
print(f"\nPrimeiras linhas:")
display(df_desp_real.head())

In [ ]:
# Identificar colunas numéricas (meses)
colunas_meses = [col for col in df_desp_real.columns if col in MESES or any(m in str(col) for m in MESES)]
print(f"Colunas de meses identificadas: {colunas_meses}")

# Alternativa: pegar colunas numéricas
colunas_num = df_desp_real.select_dtypes(include=[np.number]).columns.tolist()
print(f"Colunas numéricas: {colunas_num}")

In [ ]:
# Total de despesas por pacote
if 'Pacote' in df_desp_real.columns:
    # Calcular total por pacote
    colunas_valores = df_desp_real.select_dtypes(include=[np.number]).columns
    df_desp_real['Total'] = df_desp_real[colunas_valores].sum(axis=1)
    
    despesas_por_pacote = df_desp_real.groupby('Pacote')['Total'].sum().sort_values()
    
    print("📦 DESPESAS POR PACOTE (Realizado)")
    print("="*60)
    for pacote, valor in despesas_por_pacote.items():
        pct = (abs(valor) / abs(despesas_por_pacote).sum()) * 100
        print(f"  {pacote:<35} R$ {valor:>12,.2f} ({pct:>5.1f}%)")

In [ ]:
# Visualização: Top 10 Pacotes de Despesa
if 'Pacote' in df_desp_real.columns:
    top_pacotes = abs(despesas_por_pacote).nlargest(10)
    
    fig, ax = plt.subplots(figsize=(12, 6))
    top_pacotes.plot(kind='barh', ax=ax, color=COLORS['realizado'])
    ax.set_xlabel('Valor (R$)')
    ax.set_title('Top 10 Pacotes de Despesa (Realizado)', fontsize=14, fontweight='bold')
    ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x/1e6:.1f}M'))
    plt.tight_layout()
    plt.show()

---
## 3️⃣ Comparativo Real vs Orçado

In [ ]:
# Comparativo de despesas
colunas_valores_orc = df_desp_orc.select_dtypes(include=[np.number]).columns

total_desp_real = df_desp_real[colunas_valores].sum().sum()
total_desp_orc = df_desp_orc[colunas_valores_orc].sum().sum()

variacao = total_desp_real - total_desp_orc
variacao_pct = (variacao / abs(total_desp_orc)) * 100 if total_desp_orc != 0 else 0

print("📊 COMPARATIVO DESPESAS: REAL vs ORÇADO")
print("="*60)
print(f"  Despesas Realizadas:  R$ {total_desp_real:>15,.2f}")
print(f"  Despesas Orçadas:     R$ {total_desp_orc:>15,.2f}")
print(f"  " + "-"*50)
print(f"  Variação:             R$ {variacao:>15,.2f} ({variacao_pct:+.1f}%)")

emoji = "✅ Economia" if variacao > 0 else "⚠️ Estouro"
print(f"\n  Status: {emoji}")

In [ ]:
# Visualização do comparativo
fig, ax = plt.subplots(figsize=(8, 5))

categorias = ['Orçado', 'Realizado']
valores = [abs(total_desp_orc), abs(total_desp_real)]
cores = [COLORS['orcado'], COLORS['realizado']]

bars = ax.bar(categorias, valores, color=cores, edgecolor='black', linewidth=1.2)
ax.set_ylabel('Despesas (R$)')
ax.set_title('Despesas: Orçado vs Realizado', fontsize=14, fontweight='bold')
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x/1e6:.1f}M'))

# Adicionar valores nas barras
for bar, valor in zip(bars, valores):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 500000, 
            f'R$ {valor/1e6:.1f}M', ha='center', fontweight='bold')

# Adicionar linha de economia
ax.annotate(f'Economia: R$ {abs(variacao)/1e6:.1f}M', 
            xy=(1.2, valores[1]), fontsize=12, color='green')

plt.tight_layout()
plt.show()

---
## 4️⃣ Análise por Unidade de Negócio

In [ ]:
# Despesas por Centro de Custo/Unidade
col_unidade = 'Centro de Custo' if 'Centro de Custo' in df_desp_real.columns else 'Unidade'

if col_unidade in df_desp_real.columns:
    despesas_por_unidade = df_desp_real.groupby(col_unidade)['Total'].sum().sort_values()
    
    print(f"🏢 DESPESAS POR {col_unidade.upper()}")
    print("="*60)
    for unidade, valor in despesas_por_unidade.items():
        pct = (abs(valor) / abs(despesas_por_unidade).sum()) * 100
        print(f"  {unidade:<15} R$ {valor:>12,.2f} ({pct:>5.1f}%)")

---
## 📝 Insights Principais

### Receitas:
- Estrutura complexa em formato REPORT
- Dados divididos entre SALES e SERVICE
- Necessário unpivot para análise tabular

### Despesas:
- Formato tabular facilita análise
- Economia significativa vs orçado
- Concentração em poucos pacotes principais

### Próximos Passos:
→ Ver **Parte 3** para análise da DRE e KPIs executivos